In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('/kaggle/input/iqb213-atp/train.data')
test_data = pd.read_csv('/kaggle/input/iqb213-atp/test1.txt')
amino_acid = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','X']


In [ ]:
polarity = {'A':8.1, 'L':4.9, 'R':10.5, 'K':11.3, 'N':11.6, 'M':5.7, 'D':13, 'F':5.2, 'C':5.5, 'P':8.0, 'Q':10.5, 'S':9.2, 'E':12.3, 'T':8.6, 'G':9.0, 'W':5.4, 'H':10.4, 'Y':6.2, 'I':5.2, 'V':5.9}
hydrophobicity = {"A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5, "Q": -3.5, "E": -3.5, "G": -0.4, "H": -3.2, "I": 4.5, "L": 3.8, "K": -3.9, "M": 1.9, "F": 2.8, "P": -1.6, "S": -0.8, "T": -0.7, "W": -0.9, "Y": -1.3, "V": 4.2}
SolvationPot = {'A':0.67, 'L':1.9, 'R':-2.1, 'K':-0.57, 'N':-0.6, 'M':2.4, 'D':-1.2, 'F':2.3, 'C':0.38, 'P':1.2, 'Q':-0.22, 'S':0.01, 'E':-0.76, 'T':0.52, 'G':0, 'W':2.6, 'H':0.64, 'Y':1.6, 'I':1.9, 'V':1.5}
SurfArea = { 'A':27.8, 'L':27.6, 'R':94.7, 'K':103, 'N':60.1, 'M':33.5, 'D':60.6, 'F':25.5, 'C':15.5, 'P':51.5, 'Q':68.7, 'S':42, 'E':68.2, 'T':45, 'G':24.5, 'W':34.7, 'H':50.7, 'Y':55.2, 'I':22.8, 'V':23.7 }


In [ ]:
train_list = list(train_data.loc[:,'Amino Acid Sequence'])
for i in range(0,len(train_list)):
    train_list[i]="XXXXXXXX"+train_list[i]+"XXXXXXX"

In [ ]:
test_list = list(test_data.loc[:,'Lable'])
test_string = "XXXXXXXX"
for i in range(0,len(test_list)):
    test_string = test_string+test_list[i]
test_string = test_string + "XXXXXXXX"
test_id = list(test_data.loc[:,'ID'])
print(test_string)

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis as p

In [ ]:
train_set = []
train_op = []
for i in train_list:
    n = len(i)-16
    for j in range(0,n):
        sq = i[j:j+17]
        row = []
        for aa in sq:
            for a in amino_acid:
                if (aa.upper() == a):
                    row.append(1)

                else:
                    row.append(0)
        sq_X = sq.replace('X','')
        sq_X = sq_X.upper() 
        prot_seq = p(sq_X)
        row.append(prot_seq.charge_at_pH(7))
        row.append(prot_seq.secondary_structure_fraction()[2])
        row.append(prot_seq.protein_scale(hydrophobicity,len(sq_X),1)[0])
        row.append(prot_seq.protein_scale(polarity,len(sq_X),1)[0])
        row.append(prot_seq.protein_scale(SolvationPot,len(sq_X),1)[0])
        row.append(prot_seq.protein_scale(SurfArea,len(sq_X),1)[0])
        train_set.append(row)
        
        if (sq[8].isupper()):
            train_op.append(-1)
        else:
            train_op.append(1)
        

In [ ]:
test_set = []
for i in range(0,len(test_string)-16):
    sq=test_string[i:i+17]
    row = []
    for aa in sq:
        for a in amino_acid:
            if (aa.upper() == a):
                row.append(1)
            else:
                row.append(0)
    sq_X = sq.replace('X','')
    sq_X = sq_X.upper() 
    prot_seq = p(sq_X)
    row.append(prot_seq.charge_at_pH(7))
    row.append(prot_seq.secondary_structure_fraction()[2])
    row.append(prot_seq.protein_scale(hydrophobicity,len(sq_X),1)[0])
    row.append(prot_seq.protein_scale(polarity,len(sq_X),1)[0])
    row.append(prot_seq.protein_scale(SolvationPot,len(sq_X),1)[0])
    row.append(prot_seq.protein_scale(SurfArea,len(sq_X),1)[0])
    test_set.append(row)
    

In [ ]:
from sklearn import svm

In [ ]:
model = svm.SVC(kernel='rbf',gamma='auto',class_weight='balanced')

In [ ]:
model.fit(train_set,train_op)
pred = model.predict(test_set)
print(pred)

In [ ]:
output = [test_id,list(pred)]

In [ ]:
op = pd.DataFrame(output,index=['ID','Lable']).transpose()
print(op)
pd.DataFrame.to_csv(op,"outSVM2"+".csv",index=False)

In [ ]:
print(len(train_set))